### Essential files for Azure Storage

In [ ]:
# pip install azure-storage-file-datalake

In [164]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.storage.filedatalake import DataLakeFileClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient

### Liberaries used in code

In [165]:
import numpy as np
import urllib
import random
import json
from sklearn.model_selection import train_test_split
import pathlib
from datetime import datetime, timedelta
import requests
import pandas as pd


### Connect to Storage account

### Function to fetch

In [86]:
def fetch_data_db():
        global values
        global results
        ##following is example of unauthenticated read... should fail if the db is set to not open read
        url = "http://gapp.agverse.in/api/v1/get-telemetry-data/"
        #response = urllib.request.urlopen(url)
        #this is for authenticated reading
        url_auth=urllib.request.Request(url,headers={"username": "gapp-apis","password":"4Score&7yrsAgo"})
        response = urllib.request.urlopen(url_auth)
        data = response.read()
        values = json.loads(data)
        results = pd.DataFrame(values)
        return values

[Refer this document for complete steps](https://docs.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-directory-file-acl-python)

### Function to create a dataframe of data obtained from api link

In [68]:
def restAPI_data_to_dataFrame():
    id = []
    user_id = []
    animal_id = []
    telemetry_data = []
    mac_id = []
#     global timestamp 
    timestamp = []
    x = results.result 

    for i in x:
        id.append(i['id'])
        user_id.append(i['user_id'])
        animal_id.append(i['animal_id'])
        telemetry_data.append(i['telemetry_data'])
        mac_id.append(i['mac_id'])
        timestamp.append(i['timestamp'])
    _dict_ = {'id' : id,'user_id' : user_id,'animal_id' : animal_id,'telemetry_data' : telemetry_data,'mac_id' : mac_id,'timestamp':timestamp} 
    Data = pd.DataFrame(_dict_)
    return Data

### data_to_storage_acc

In [183]:
def data_to_storage_acc():
    service = BlobServiceClient(account_url="https://sampleagverse.blob.core.windows.net/", credential=storage_account_key)
    
    blob = BlobClient.from_connection_string(conn_str=my_connection_string, container_name="sampleagverse", blob_name=("Data "+str(datetime.datetime.now())))

    fetch_data_db()
    
    restAPI_data_to_dataFrame().to_csv('Data.csv')

    with open("./Data.csv", "rb") as data:
        blob.upload_blob(data)
    
    print("Data uploaded successfully")

# Main 

### Connect to storage account

In [180]:
storage_account_name = 'sampleagverse'
storage_account_key = 'Pg2WyUvKwJeG/mP/KO1zeBM/90N0+hiaCiQgRHDbaVLfXpvd6gSOsHhBSQLFipCW3dE3vpVcmYJJDTkR1lw2mw=='
my_connection_string = 'DefaultEndpointsProtocol=https;AccountName=sampleagverse;AccountKey=Pg2WyUvKwJeG/mP/KO1zeBM/90N0+hiaCiQgRHDbaVLfXpvd6gSOsHhBSQLFipCW3dE3vpVcmYJJDTkR1lw2mw==;EndpointSuffix=core.windows.net'
# initialize_storage_account(storage_account_name,storage_account_key)

In [184]:
import datetime

data_to_storage_acc()

Data uploaded successfully


In [172]:
 dataFromApi = restAPI_data_to_dataFrame()


In [173]:
dataFromApi

,id,user_id,animal_id,telemetry_data,mac_id,timestamp
0,1,20,22,56 84 33 56 -21 -104,FD:7E:37:9E:A6:A9,25/12/2020 22:16:36
1,2,20,22,56 84 33 56 -21 -104,FD:7E:37:9E:A6:A9,25/12/2020 22:16:36
2,3,20,22,56 104 33 12 -21 -104,FD:7E:37:9E:A6:A9,25/12/2020 22:16:46
3,4,20,22,56 104 33 12 -21 -104,FD:7E:37:9E:A6:A9,25/12/2020 22:16:46
4,5,20,21,40 -104 27 -16 42 68,E1:4C:CA:A7:CD:C9,25/12/2020 22:16:51
...,...,...,...,...,...,...
15657,15658,20,6,"FFFF,FFFF,FFFF",CA:FB:63:73:E5:35,1620722908
15658,15659,20,7,"0688,00D4,3FF8",DD:70:7B:28:82:CC,1620722909
15659,15660,20,6,"FFFF,FFFF,FFFF",CA:FB:63:73:E5:35,1620722913
15660,15661,20,7,"0618,00DC,3FE0",DD:70:7B:28:82:CC,1620722914
